In [34]:
import tkinter as tk
from tkinter import filedialog, messagebox
import pandas as pd
import os

credit_keywords = {
    "zelle": "Gross Trucking Income",
    "deposit": "Gross Trucking Income",
    "payment received": "Gross Trucking Income",
    "refund": "Reimbursement",
    "rebate": "Reimbursement",
    "mobile deposit": "Gross Trucking Income",
    "purchase return" : "return",
}

credit_return_keywords = {
    "fontana ca" : "Subhaulers & Carriers"
}

debit_keywords = {
    "zelle": "Gross Trucking Income",
    "fuel": "Fuel Cost",
    "pump": "Fuel Cost",
    "gas": "Fuel Cost",
    "insurance": "Insurance",
    "advance": "Loan or Advance",
    "loan": "Loan or Advance",
    "restaurant": "Food & Dining",
    "bar": "Entertainment",
    "grill": "Food & Dining",
    "hotel": "Lodging",
    "subscription": "Subscription",
    "grocery": "Groceries",
    "prepass": "Tolls and Parking",

    
    "purchase authorized": "Subhaulers & Carriers",
    "check" : "Truck Maintenance Cost",
    

    # Companies
    "pathward na" : "Liability Insurance",
    "ibex " : "Liability Insurance",

    # Licences and Permits
    "jst registration" : "Business Licenses and Permits",
    "unified carrier" : "Business Licenses and Permits",

    # Medical
    "drug testing": "Drug Testing",

    # Taxes
    "cdtfa" : "CDTFA",
    "tax" : "Taxes",

    # Bank
    "monthly service fee" : "Bank Service Charges"
}
debit_check_keywords = {
    "check" : "Truck Maintenance Cost",
    "cashed" : "Truck Maintenance Cost",
    "deposited" : "Truck Maintenance Cost",
    "overdraft fee": "Bank Service Charges",
}


def process_excel(input_path, output_folder):
    try:
        df = pd.read_excel(input_path)
        categories = []

        for _, row in df.iterrows():
            amount = row["Amount"]
            desc = row["Description"]

            if amount > 0:
                category = categorize_credit(desc)
            else:
                category = categorize_debit(desc)

            # if not category:
            #     category = lookup_vendor_web(desc)

            categories.append(category)

        df["Category"] = categories

        output_path = os.path.join(output_folder, f"processed_{os.path.basename(input_path)}")
        df.to_excel(output_path, index=False)
    except Exception as e:
        messagebox.showerror("Processing Error", f"Failed processing:\n{input_path}\n\nError:\n{str(e)}")


def categorize_credit(description):
    desc = str(description).lower()
    for key, value in credit_keywords.items():
        if key in desc:
            if value == "return":
                for key1, value1 in credit_return_keywords.items():
                    if key1 in desc:
                        return value1
            return value
            
    return None

def categorize_debit(description):
    desc = str(description).lower()
    for key, value in debit_keywords.items():
        if key in desc:
            if "check" in desc:
                for key1, value1 in debit_check_keywords.items():
                    if desc.startswith(key1):
                        return value1
            return value
    return None

def select_files():
    files = filedialog.askopenfilenames(filetypes=[("Excel files", "*.xlsx *.xls")])
    if files:
        global selected_input_files
        selected_input_files = list(files)  # store real list
        input_files_display_var.set("; ".join(files))  # only for display

def select_output_folder():
    folder = filedialog.askdirectory()
    if folder:
        output_folder_var.set(folder)

def run_processing():
    if not selected_input_files or not output_folder_var.get():
        messagebox.showerror("Error", "Please select input files and output folder.")
        return
    for file_path in selected_input_files:
        process_excel(file_path, output_folder_var.get())
    messagebox.showinfo("Success", "Processing completed!")

root = tk.Tk()
root.title("Excel Processor")

# Variables
selected_input_files = []  # real list of files # JUST for Debugging
selected_input_files.append("C:/Users/TCE/Downloads/NonStop.xlsx")  # JUST for Debugging
input_files_display_var = tk.StringVar()  # shown in entry
input_files_display_var.set("; ".join(selected_input_files))  # JUST for Debugging
output_folder_var = tk.StringVar()
output_folder_var.set("C:/Users/TCE/Downloads")  # JUST for Debugging

# Widgets
tk.Button(root, text="Select Excel Files", command=select_files).pack(pady=5)
tk.Entry(root, textvariable=input_files_display_var, width=80, state="readonly").pack(pady=5)

tk.Button(root, text="Select Output Folder", command=select_output_folder).pack(pady=5)
tk.Entry(root, textvariable=output_folder_var, width=80, state="readonly").pack(pady=5)

tk.Button(root, text="Run", command=run_processing).pack(pady=20)

root.mainloop()


In [5]:
import pandas as pd

# Set pandas display options to show full width
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.width', 0)            # No line width restriction
pd.set_option('display.max_colwidth', None)  # Show full content inside each column

# Load file
file_path = "input.xlsx"
df = pd.read_excel(file_path, sheet_name="Sheet1", header=0)

# Fix unnamed columns
new_columns = []
for idx, col in enumerate(df.columns):
    if "Unnamed" in str(col) or pd.isna(col):
        new_columns.append(f"Column_{idx+1}")
    else:
        new_columns.append(col)

df.columns = new_columns

# Display
print(df.head(50))

# Rows and columns count
print(f"Number of rows: {df.shape[0]}")
print(f"Number of columns: {df.shape[1]}")


   Column_1   Column_2 Column_3  Column_4  Column_5     Type  Column_7  \
0       NaN  BOA #7483      NaN       NaN       NaN      NaN       NaN   
1       NaN        NaN      NaN       NaN       NaN    Check       NaN   
2       NaN        NaN      NaN       NaN       NaN    Check       NaN   
3       NaN        NaN      NaN       NaN       NaN    Check       NaN   
4       NaN        NaN      NaN       NaN       NaN    Check       NaN   
5       NaN        NaN      NaN       NaN       NaN    Check       NaN   
6       NaN        NaN      NaN       NaN       NaN    Check       NaN   
7       NaN        NaN      NaN       NaN       NaN    Check       NaN   
8       NaN        NaN      NaN       NaN       NaN    Check       NaN   
9       NaN        NaN      NaN       NaN       NaN    Check       NaN   
10      NaN        NaN      NaN       NaN       NaN    Check       NaN   
11      NaN        NaN      NaN       NaN       NaN    Check       NaN   
12      NaN        NaN      NaN       

In [6]:
import pandas as pd

file_path = "input.xlsx"

# Now load correctly from "Sheet2"
df = pd.read_excel(file_path, sheet_name="Sheet1", header=0)

# Rename unnamed columns properly
new_columns = []
for idx, col in enumerate(df.columns):
    if "Unnamed" in str(col) or pd.isna(col):
        new_columns.append(f"Column_{idx+1}")
    else:
        new_columns.append(col)
df.columns = new_columns

# Initialize debit and credit dictionaries
from collections import defaultdict
import re

debit_keywords = defaultdict(dict)
credit_keywords = defaultdict(dict)

# Helper function to clean and extract a short key from Memo
def extract_key_from_memo(memo):
    if pd.isna(memo):
        return "Unknown"
    cleaned_memo = re.sub(r'[^a-zA-Z0-9\s]', '', memo)
    words = cleaned_memo.strip().split()
    key = " ".join(words[:3]) if words else "Unknown"
    return key

# Keep track of the current category from Column_2
current_category = None

# Loop through rows
for _, row in df.iterrows():
    if not pd.isna(row["Column_2"]):
        current_category = row["Column_2"]

    memo = row.get("Memo", "")
    key = extract_key_from_memo(memo)

    debit_value = row.get("Debit", None)
    credit_value = row.get("Credit", None)

    if pd.notna(debit_value):
        debit_keywords[current_category][key] = debit_value
    elif pd.notna(credit_value):
        credit_keywords[current_category][key] = credit_value

# Convert defaultdicts to normal dicts
debit_keywords = dict(debit_keywords)
credit_keywords = dict(credit_keywords)

(debit_keywords, credit_keywords)


({'BOA #7483': {'WIRE TYPEBOOK IN': 81505.36,
   'WIRE TYPEWIRE IN': 25000.0,
   'SAIAINC DESSAIAINC ID80148': 19157.6,
   'RETURNED CHECK FDES': 2566.64,
   'LOVES TRAVEL STO': 50.02,
   'SAIAINC DESSAIAINC ID80614': 19056.8,
   'TriumphPay CR DESTriumphPay': 400.0,
   'BKOFAMERICA MOBILE 0131': 2290.0,
   'SAIAINC DESSAIAINC ID81903': 22495.2,
   'BKOFAMERICA MOBILE 0205': 1543.73,
   'SAIAINC DESSAIAINC ID82432': 18720.8,
   'BKOFAMERICA MOBILE 0213': 2560.0,
   'BKOFAMERICA MOBILE 0214': 117.45,
   'SAIAINC DESSAIAINC ID82851': 14912.8,
   'Counter Credit': 4250.0,
   'SAIAINC DESSAIAINC ID83351': 14963.2,
   'GOOGLE DESACCTVERIFY IDUS003ZLQAQ': 0.87,
   'BKOFAMERICA MOBILE 0308': 112.28,
   'SAIAINC DESSAIAINC ID84414': 7481.6,
   'BKOFAMERICA MOBILE 0321': 3405.6,
   'SAIAINC DESSAIAINC ID85376': 11474.4,
   'SAIAINC DESSAIAINC ID86010': 11474.4,
   'BKOFAMERICA MOBILE 0401': 3900.0,
   'SAIAINC DESSAIAINC ID86463': 11440.8,
   'SAIAINC DESSAIAINC ID86979': 15181.6,
   'Prfd Rwds

In [30]:
import pandas as pd
import os

def process_excel(input_path, output_folder):
    try:
        df = pd.read_excel(input_path)
        categories = []

        for _, row in df.iterrows():
            amount = row["Amount"]
            desc = row["Description"]

            if amount > 0:
                category = categorize_credit(desc)
            else:
                category = categorize_debit(desc)

            # if not category:
            #     category = lookup_vendor_web(desc)

            categories.append(category)

        df["Category"] = categories

        output_path = os.path.join(output_folder, f"processed_{os.path.basename(input_path)}")
        df.to_excel(output_path, index=False)
    except Exception as e:
        messagebox.showerror("Processing Error", f"Failed processing:\n{input_path}\n\nError:\n{str(e)}")


def categorize_credit(description):
    desc = str(description).lower()
    for key, value in credit_keywords.items():
        if key in desc:
            if value == "return":
                for key1, value1 in credit_return_keywords.items():
                    if key1 in desc:
                        return value1
            return value
            
    return None

def categorize_debit(description):
    desc = str(description).lower()
    for key, value in debit_keywords.items():
        if key in desc:
            if "check" in desc:
                for key1, value1 in debit_check_keywords.items():
                    if desc.startswith(key1):
                        return value1
            return value
    return None




file_path = "input.xlsx"

# Now load correctly from "Sheet2"
df = pd.read_excel(file_path, sheet_name="Sheet1", header=0)

# Rename unnamed columns
new_columns = []
for idx, col in enumerate(df.columns):
    if "Unnamed" in str(col) or pd.isna(col):
        new_columns.append(f"Column_{idx+1}")
    else:
        new_columns.append(col)
df.columns = new_columns

# Initialize FLAT dictionaries
debit_keywords = {}
credit_keywords = {}

# Helper: extract clean short key from Memo
def extract_key_from_memo(memo):
    if pd.isna(memo):
        return "Unknown"
    cleaned_memo = re.sub(r'[^a-zA-Z0-9\s]', '', memo)
    words = cleaned_memo.strip().split()
    key = " ".join(words[:3]) if words else "Unknown"
    return key

# Track current category
current_category = None

# Loop through the dataframe
for i, row in df.iterrows():
    # new_category = row.get("Column_2", None)
    # if not pd.isna(new_category):
    #     if ("BOA #" in new_category) or ("Total " in new_category) or ("FCB Bank" in new_category):
    #         continue
    #     current_category = new_category
    col2_value = row.get("Column_2", None)

    if not pd.isna(col2_value):
        # Only update current_category if the value is not 'useless'
        if ("BOA #" not in col2_value) and ("Total " not in col2_value) and ("FCB Bank" not in col2_value) or ("FCB Visa CC" not in col2_value) or ("BOA CC" not in col2_value):
            current_category = col2_value  # Only update when valid!
    if current_category is None:
        continue

    memo = row.get("Memo", "")
    key = extract_key_from_memo(memo)

    debit_value = row.get("Debit", None)
    credit_value = row.get("Credit", None)

    if pd.notna(debit_value):
        debit_keywords[key] = current_category
    elif pd.notna(credit_value):
        credit_keywords[key] = current_category

# print((debit_keywords, credit_keywords))
print(len(debit_keywords.keys()), len(credit_keywords.keys()))
input_file = "C:/develop/FinanceClassifier/in.xlsx"
output_folder = "C:/develop/FinanceClassifier"
process_excel(input_file, output_folder)



2821 2749
